In [1]:
# Tokenizing text sample

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Total len {len(raw_text)}")
print(raw_text[:90])



Total len 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so i


In [2]:
# Simple sample how to tokenize with re
import re

text = "Hello--, this: is some sort of sample TEXT. Let's  tokenze it!?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)

# removing whitespce chars
result = [it for it in result if it.strip()]
print(result)

['Hello', '--', ',', 'this', ':', 'is', 'some', 'sort', 'of', 'sample', 'TEXT', '.', 'Let', "'", 's', 'tokenze', 'it', '!', '?']


In [3]:
# Applying basic tokenizer to all text
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()] # twice because fist is check for empty, second for clean
print(len(preprocessed))
print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [4]:
# Creating dict
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f"vocab_size: {vocab_size}")

# print first values in it
vocab = {token:integer for integer, token in enumerate(all_words)}
print(*list(vocab.items())[:50],sep="\n")

vocab_size: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)


In [5]:
# Tokenizer class

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV1(vocab=vocab)
test = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(test)
print(tokenizer.decode(ids))

# But it will not work with words are not presented in vocab
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


KeyError: 'Hello'

In [7]:
# 2.4 Adding special context tokens
all_tokens = sorted(list(set(preprocessed))) 
all_tokens.extend(["<|endoftext|>", "<|unk|>"]) 
vocab = {token:integer for integer,token in enumerate(all_tokens)} 
print(len(vocab.items()))

print(list(vocab.items())[-5:])


1132
[('younger', 1127), ('your', 1128), ('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]


In [12]:
# Modifying class to support unk words
class SimpleTokenizerV2(SimpleTokenizerV1):
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Logic for unknown words
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>"
            for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

tokenizer2 = SimpleTokenizerV2(vocab=vocab)
test1 = "Hello, do you like tea?"
test2 = "In the sunlit terraces of the palace."
text1 = " <|endoftext|> ".join((test1, test2))
ids2 = tokenizer2.encode(text1)
print(tokenizer2.decode(ids2))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
